# CUDA - Floyd Warshall Algorithm

### Uninstall CUDA
Uninstall version 11

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

### Install CUDA
Install version 9.2

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

### Verify that CUDA has been installed successfully

In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


### Install and load the $nvcc$ plugin

In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [ ]:
%load_ext nvcc_plugin

### Execute the Program

In [ ]:
%%cu
#include <stdio.h>
#include <stdlib.h>
#include <sys/time.h>

#define vertices 2500
#define THREADS_PER_BLOCK 8

__global__ void floydWarshallGPU(int *dist, int k){
    int i = (blockIdx.x * blockDim.x) + threadIdx.x;
    int j = (blockIdx.y * blockDim.y) + threadIdx.y;

    if(i < vertices && j < vertices) {
        int idx = i*vertices+j;
        int idx1 = k*vertices+j;
        int idx2 = i*vertices+k;

        __syncthreads();

        if(dist[idx] > dist[idx1] + dist[idx2]) {
            dist[idx] = dist[idx1] + dist[idx2];
        }

        __syncthreads();   
    }
}

void floydWarshall(int adj_matrix[vertices][vertices]){
    int *distances;
    int *device_distances;
    int *ans;

    size_t bytes = vertices*vertices*sizeof(int);

    distances = (int *)malloc(bytes);
    ans = (int *)malloc(bytes);

    cudaMalloc(&device_distances, bytes);

    for(int i=0;i<vertices;i++){
        for(int j=0;j<vertices;j++){
            distances[i*vertices+j] = adj_matrix[i][j];
        }
    }

    cudaMemcpy(device_distances, distances, bytes, cudaMemcpyHostToDevice);

    dim3 block((vertices/THREADS_PER_BLOCK) + 1, (vertices/THREADS_PER_BLOCK) + 1, 1);
    dim3 threadsPerBlock(THREADS_PER_BLOCK, THREADS_PER_BLOCK, 1);

    for(int k=0;k<vertices;k++){
        floydWarshallGPU<<<block, threadsPerBlock>>>(device_distances, k);
    }

    cudaDeviceSynchronize();

    cudaMemcpy(ans, device_distances, bytes, cudaMemcpyDeviceToHost);
}

int main(int argc, char** argv){
    struct timeval TimeValue_Start;
    struct timezone TimeZone_Start;
    struct timeval TimeValue_Final;
    struct timezone TimeZone_Final;
    long time_start, time_end;
    double time_overhead;

    static int adj_matrix[vertices][vertices];

    for(int i=0;i<vertices;i++){
        for(int j=0;j<vertices;j++){
            if(i==j)
            adj_matrix[i][j] = 0;
            else
            adj_matrix[i][j] = rand()%10000;
        }
    }

    gettimeofday(&TimeValue_Start, &TimeZone_Start);
    floydWarshall(adj_matrix);
    gettimeofday(&TimeValue_Final, &TimeZone_Final);
    time_start = TimeValue_Start.tv_sec * 1000000 + TimeValue_Start.tv_usec;
    time_end = TimeValue_Final.tv_sec * 1000000 + TimeValue_Final.tv_usec;
    time_overhead = (time_end - time_start)/1000000.0;
    printf("\n\n\t\t Time in Seconds (T) : %lf\n",time_overhead);

    return 0;
}



		 Time in Seconds (T) : 2.156729

